In [1]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file


In [2]:
client : OpenAI = OpenAI()

## Step 1: Upload the file and Create an Assistant

In [3]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("basic_maths_book.pdf", "rb"),
  purpose='assistants'
)

# print(file)
# file_id = id='file-x1dToGcyA2PuuZwoo8aNEKIV'

In [4]:
instructions="""You are a student support chatbot. Use your knowledge base to best respond to student maths queries.
Instructions:
- Give a short and terse answer to the following question.
- solve the mathamatical problem. student can gives you the questions mathematical problem.
- also you can give the explanation of the answer. answer gives in mathamatical form.
- give the pg number of the answer in the pdf is define.
  
"""


In [6]:
assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  model="gpt-3.5-turbo-1106",
  instructions=instructions,
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

## Step 2: Create a Thread

In [7]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_iDLcbwC8kYnALX14aG7IBFE3', created_at=1715157149, metadata={}, object='thread')


## Step 3: Add a Message to a Thread

In [23]:
# from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""tell me about pakistan. in just 2 words, and also give the pg number of the answer in the pdf is define.""",
)

## Step 4: Run the Assistant

In [24]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)

## Step 5: Check the Run status

In [25]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)

Run(id='run_oEGiHewtxoQJBrG8Fpmprr22', assistant_id='asst_cUaN5uQge8u7KfzT5arOVmPN', cancelled_at=None, completed_at=None, created_at=1715159326, expires_at=1715159926, failed_at=None, file_ids=['file-S9rbrhNtTfHbyr0Iqdh101zl'], instructions='Please address the user as Pakistani. The user is the student of PIAIC.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1715159326, status='in_progress', thread_id='thread_iDLcbwC8kYnALX14aG7IBFE3', tools=[RetrievalTool(type='retrieval')], usage=None, temperature=1.0, top_p=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto')


## Step 6: Display the Assistant's Response

In [27]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)

user:  What is the answer of 30+279? and explain it.
assistant: The answer to 30 + 279 is 309【7†source】. It seems that this is part of a programmed text designed to help learn and apply the principles of simple addition, subtraction, multiplication, and division of whole numbers and common fractions. The program involves studying the text and then testing oneself immediately by answering questions, filling in blanks, and performing fundamental mathematical computations. It is aimed at helping individuals apply these principles effectively.
user:  What is the answer of 2,175 + 24 =______? and also give the pg number of the answer in the pdf is define.
assistant: The answer to 2,175 + 24 is 2,199【12†source】. This information can be found on page 5 of the PDF.
user:  What is the answer of 2 + 2 =______? is this sol present in pdf? and also give the pg number of the answer in the pdf is define.
assistant: The answer to 2 + 2 is 4【17†source】. This information can be found on page 46 of the 